<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Pre-Trained Models</font></h1>


## Objective


In this lab, you will learn how to leverage pre-trained models to build image classifiers instead of building a model from scratch.


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3> 
    
1. <a href="#item31">Import Libraries and Packages</a>
2. <a href="#item32">Download Data</a>  
3. <a href="#item33">Define Global Constants</a>  
4. <a href="#item34">Construct ImageDataGenerator Instances</a>  
5. <a href="#item35">Compile and Fit Model</a>

</font>
    
</div>


<a id='item31'></a>


## Import Libraries and Packages


Let's start the lab by importing the libraries that we will be using in this lab.


First, we will import the ImageDataGenerator module since we will be leveraging it to train our model in batches.


In [1]:
from keras.preprocessing.image import ImageDataGenerator

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In this lab, we will be using the Keras library to build an image classifier, so let's download the Keras library.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Finally, we will be leveraging the ResNet50 model to build our classifier, so let's download it as well.


In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

<a id='item32'></a>


## Download Data


For your convenience, I have placed the data on a server which you can retrieve easily using the **wget** command. So let's run the following line of code to get the data. Given the large size of the image dataset, it might take some time depending on your internet speed.


In [4]:
## get the data
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

And now if you check the left directory pane, you should see the zipped file _concrete_data_week3.zip_ appear. So, let's go ahead and unzip the file to access the images. Given the large number of images in the dataset, this might take a couple of minutes, so please be patient, and wait until the code finishes running.


In [5]:
#!unzip concrete_data_week3.zip

Now, you should see the folder _concrete_data_week3_ appear in the left pane. If you open this folder by double-clicking on it, you will find that it contains two folders: _train_ and _valid_. And if you explore these folders, you will find that each contains two subfolders: _positive_ and _negative_. These are the same folders that we saw in the labs in the previous modules of this course, where _negative_ is the negative class and it represents the concrete images with no cracks and _positive_ is the positive class and it represents the concrete images with cracks.


**Important Note**: There are thousands and thousands of images in each folder, so please don't attempt to double click on the _negative_ and _positive_ folders. This may consume all of your memory and you may end up with a **50\*** error. So please **DO NOT DO IT**.


<a id='item33'></a>


## Define Global Constants


Here, we will define constants that we will be using throughout the rest of the lab. 

1.  We are obviously dealing with two classes, so _num_classes_ is 2. 
2.  The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3.  We will be training and validating the model using batches of 100 images.


In [6]:
num_classes = 2
image_resize = 224
batch_size_training = 300
batch_size_validation = 300

<a id='item34'></a>


## Construct ImageDataGenerator Instances


In order to instantiate an ImageDataGenerator instance, we will set the **preprocessing_function** argument to _preprocess_input_ which we imported from **keras.applications.resnet50** in order to preprocess our images the same way the images used to train ResNet50 model were processed.


In [7]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

Next, we will use the _flow_from_directory_ method to get the training images as follows:


In [8]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


**Your Turn**: Use the _flow_from_directory_ method to get the validation images and assign the result to **validation_generator**.


In [9]:
## Type your answer here
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 10001 images belonging to 2 classes.


Double-click **here** for the solution.

<!-- The correct answer is:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')
-->


<a id='item35'></a>


## Build, Compile and Fit Model


In this section, we will start building our model. We will use the Sequential model class from Keras.


In [10]:
model = Sequential()

Next, we will add the ResNet50 pre-trained model to out model. However, note that we don't want to include the top layer or the output layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument _include_top_ and set it to **False**.


In [11]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

Then, we will define our output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.


In [12]:
model.add(Dense(num_classes, activation='softmax'))

You can access the model's layers using the _layers_ attribute of our model object. 


In [13]:
model.layers

You can see that our model is composed of two sets of layers. The first set is the layers pertaining to ResNet50 and the second set is a single layer, which is our Dense layer that we defined above.


You can access the ResNet50 layers by running the following:


In [14]:
model.layers[0]

Since the ResNet50 model has already been trained, then we want to tell our model not to bother with training the ResNet part, but to train only our dense output layer. To do that, we run the following.


In [15]:
model.layers[0].trainable = False

And now using the _summary_ attribute of the model, we can see how many parameters we will need to optimize in order to train the output layer.


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


Next we compile our model using the **adam** optimizer.


In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Before we are able to start the training process, with an ImageDataGenerator, we will need to define how many steps compose an epoch. Typically, that is the number of images divided by the batch size. Therefore, we define our steps per epoch as follows:


In [18]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 1

Finally, we are ready to start training our model. Unlike a conventional deep learning training were data is not streamed from a directory, with an ImageDataGenerator where data is augmented in batches, we use the **fit_generator** method.


In [31]:
base_model = ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
    pooling='avg')  # Do not include the ImageNet classifier at the top.

# Freeze the base model
base_model.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
model = keras.Model(inputs, outputs)

The following Variables were used a Lambda layer's call (tf.nn.convolution_53), but
are not present in its tracked objects:
  <tf.Variable 'conv1_conv/kernel:0' shape=(7, 7, 3, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_53), but
are not present in its tracked objects:
  <tf.Variable 'conv1_conv/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_53), but
are not present in its tracked objects:
  <tf.Variable 'conv1_bn/gamma:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv1_bn/be

The following Variables were used a Lambda layer's call (tf.nn.bias_add_59), but
are not present in its tracked objects:
  <tf.Variable 'conv2_block2_2_conv/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_59), but
are not present in its tracked objects:
  <tf.Variable 'conv2_block2_2_bn/gamma:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2_block2_2_bn/beta:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_60), but
are not present in its tracked objects:
  <tf.Variab

The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_65), but
are not present in its tracked objects:
  <tf.Variable 'conv3_block1_2_bn/gamma:0' shape=(128,) dtype=float32>
  <tf.Variable 'conv3_block1_2_bn/beta:0' shape=(128,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_66), but
are not present in its tracked objects:
  <tf.Variable 'conv3_block1_3_conv/kernel:0' shape=(1, 1, 128, 512) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_66), but
are not present in its tracked object

The following Variables were used a Lambda layer's call (tf.nn.convolution_72), but
are not present in its tracked objects:
  <tf.Variable 'conv3_block3_2_conv/kernel:0' shape=(3, 3, 128, 128) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_72), but
are not present in its tracked objects:
  <tf.Variable 'conv3_block3_2_conv/bias:0' shape=(128,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_72), but
are not present in its tracked objects:
  <tf.Variable 'conv3_block3_2_bn/gamma:0' shape=(128,) dtype=floa

The following Variables were used a Lambda layer's call (tf.nn.bias_add_78), but
are not present in its tracked objects:
  <tf.Variable 'conv4_block1_2_conv/bias:0' shape=(256,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_78), but
are not present in its tracked objects:
  <tf.Variable 'conv4_block1_2_bn/gamma:0' shape=(256,) dtype=float32>
  <tf.Variable 'conv4_block1_2_bn/beta:0' shape=(256,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_79), but
are not present in its tracked objects:
  <tf.Var

The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_84), but
are not present in its tracked objects:
  <tf.Variable 'conv4_block3_1_bn/gamma:0' shape=(256,) dtype=float32>
  <tf.Variable 'conv4_block3_1_bn/beta:0' shape=(256,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_85), but
are not present in its tracked objects:
  <tf.Variable 'conv4_block3_2_conv/kernel:0' shape=(3, 3, 256, 256) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_85), but
are not present in its tracked object

The following Variables were used a Lambda layer's call (tf.nn.convolution_91), but
are not present in its tracked objects:
  <tf.Variable 'conv4_block5_2_conv/kernel:0' shape=(3, 3, 256, 256) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_91), but
are not present in its tracked objects:
  <tf.Variable 'conv4_block5_2_conv/bias:0' shape=(256,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_91), but
are not present in its tracked objects:
  <tf.Variable 'conv4_block5_2_bn/gamma:0' shape=(256,) dtype=floa

The following Variables were used a Lambda layer's call (tf.nn.bias_add_97), but
are not present in its tracked objects:
  <tf.Variable 'conv5_block1_2_conv/bias:0' shape=(512,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_97), but
are not present in its tracked objects:
  <tf.Variable 'conv5_block1_2_bn/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'conv5_block1_2_bn/beta:0' shape=(512,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_98), but
are not present in its tracked objects:
  <tf.Var

The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_103), but
are not present in its tracked objects:
  <tf.Variable 'conv5_block3_1_bn/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'conv5_block3_1_bn/beta:0' shape=(512,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_104), but
are not present in its tracked objects:
  <tf.Variable 'conv5_block3_2_conv/kernel:0' shape=(3, 3, 512, 512) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_104), but
are not present in its tracked obj

In [32]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
tf.compat.v1.pad_2 (TFOpLambda) (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.nn.convolution_53 (TFOpLambd (None, 112, 112, 64) 0           tf.compat.v1.pad_2[0][0]         
__________________________________________________________________________________________________
tf.nn.bias_add_53 (TFOpLambda)  (None, 112, 112, 64) 0           tf.nn.convolution_53[0][0]       
______________________________________________________________________________________________

In [34]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 1

In [35]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

301/301 [==============================] - 3224s 11s/step - loss: 0.0434 - accuracy: 0.9899 - val_loss: 0.0056 - val_accuracy: 0.9992


Now that the model is trained, you are ready to start using it to classify images.


Since training can take a long time when building deep learning models, it is always a good idea to save your model once the training is complete if you believe you will be using the model again later. You will be using this model in the next module, so go ahead and save your model.


In [36]:
model.save('classifier_resnet_model.h5')

TypeError: Layer tf.nn.convolution_53 was passed non-JSON-serializable arguments. Arguments had types: {'filters': <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>, 'strides': [<class 'int'>, <class 'int'>], 'padding': <class 'str'>, 'dilations': [<class 'int'>, <class 'int'>], 'data_format': <class 'str'>, 'name': <class 'str'>}. They cannot be serialized out when saving the model.

Now, you should see the model file _classifier_resnet_model.h5_ apprear in the left directory pane.


### Thank you for completing this lab!

This notebook was created by Alex Aklson. I hope you found this lab interesting and educational.


This notebook is part of a course on **Coursera** called _AI Capstone Project with Deep Learning_. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week3_LAB1).


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                                          |
| ----------------- | ------- | ---------- | ----------------------------------------------------------- |
| 2020-09-18        | 2.0     | Shubham    | Migrated Lab to Markdown and added to course repo in GitLab |


<hr>

Copyright © 2020 [IBM Developer Skills Network](https://cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork-20647850&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork-20647850&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork-20647850&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork-20647850&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork-20647850&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork-20647850&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
